In [5]:
!hf auth whoami

mohittalniya
orgs:  cerebras-llama4-hackathon


## Download the Dataset using dataset



In [11]:
from datasets import load_dataset
ds = load_dataset("neo4j/text2cypher-2024v1")
train_samples = ds['train']
test_samples = ds['test']

Generating train split:   0%|          | 0/39554 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4833 [00:00<?, ? examples/s]

In [12]:
for i in range(10):
    print(train_samples[i])

{'question': 'Which 3 countries have the most entities linked as beneficiaries in filings?', 'schema': 'Node properties:\n- **Country**\n  - `location`: POINT \n  - `code`: STRING Example: "AFG"\n  - `name`: STRING Example: "Afghanistan"\n  - `tld`: STRING Example: "AF"\n- **Filing**\n  - `begin`: DATE_TIME Min: 2000-02-08T00:00:00Z, Max: 2017-09-05T00:00:00Z\n  - `end`: DATE_TIME Min: 2000-02-08T00:00:00Z, Max: 2017-11-03T00:00:00Z\n  - `originator_bank_id`: STRING Example: "cimb-bank-berhad"\n  - `sar_id`: STRING Example: "3297"\n  - `beneficiary_bank`: STRING Example: "Barclays Bank Plc"\n  - `filer_org_name_id`: STRING Example: "the-bank-of-new-york-mellon-corp"\n  - `originator_bank_country`: STRING Example: "Singapore"\n  - `beneficiary_bank_country`: STRING Example: "United Kingdom"\n  - `filer_org_name`: STRING Example: "The Bank of New York Mellon Corp."\n  - `originator_iso`: STRING Example: "SGP"\n  - `beneficiary_bank_id`: STRING Example: "barclays-bank-plc-london-england-g

In [13]:
print(ds['train'].features)

{'question': Value(dtype='string', id=None), 'schema': Value(dtype='string', id=None), 'cypher': Value(dtype='string', id=None), 'data_source': Value(dtype='string', id=None), 'instance_id': Value(dtype='string', id=None), 'database_reference_alias': Value(dtype='string', id=None)}


In [ ]:
import dspy


openai_api_key = "YOUR_OPENAI_API_KEY"
llm = dspy.OpenAI(model='gpt-3.5-turbo', api_key=openai_api_key, max_tokens=100)
dspy.settings.configure(lm=llm)


class Translate(dspy.Signature):
    """Translates text into a target language."""
    
    source_text = dspy.InputField(desc="The text to be translated.")
    target_language = dspy.InputField(desc="The desired output language.")
    translated_text = dspy.OutputField(desc="The text after translation.")


def translate_batch_dspy(questions, target_languages):
    """
    Translates a batch of questions into multiple target languages using DSPy.
    """
    translator = dspy.Predict(Translate)
    
    translations = {lang: [] for lang in target_languages}
    
    for question in questions:
        for lang in target_languages:
            response = translator(source_text=question, target_language=lang)
            translations[lang].append(response.translated_text)
            
    return translations


questions_batch = [sample['question'] for sample in train_samples]
target_languages = ['Tamil', 'Hindi']


translations_dspy = translate_batch_dspy(questions_batch, target_languages)
print(translations_dspy)

In [ ]:
class GenerateCypher(dspy.Signature):
    """Generates Cypher query from a question and schema in a specific language."""
    
    question = dspy.InputField(desc="The question in the target language.")
    schema = dspy.InputField(desc="The database schema.")
    cypher = dspy.OutputField(desc="The generated Cypher query.")


def generate_cypher_batch_dspy(samples, language):
    """
    Generates Cypher queries for each sample in the specified language using DSPy.
    """
    cypher_generator = dspy.Predict(GenerateCypher)
    cypher_queries = []

    for sample in samples:
        question = sample['question']
        schema = sample['schema']
        # If translations_dspy is available, use translated question
        if 'translations_dspy' in globals() and language in translations_dspy:
            idx = samples.index(sample)
            question = translations_dspy[language][idx]
        response = cypher_generator(question=question, schema=schema)
        cypher_queries.append(response.cypher)
    
    return cypher_queries

# Example usage for Tamil
cypher_queries_tamil = generate_cypher_batch_dspy(train_samples, 'Tamil')
print(cypher_queries_tamil[:5])

In [ ]:
class ExecuteCypher(dspy.Signature):
    """Executes a Cypher query against Neo4j and returns the result."""
    
    cypher = dspy.InputField(desc="The Cypher query to execute.")
    result = dspy.OutputField(desc="The result of the query execution.")

def execute_and_match(samples, cypher_queries, neo4j_driver):
    """
    Executes generated Cypher queries against Neo4j and matches the result using DSPy.
    """
    executor = dspy.Predict(ExecuteCypher)
    results = []
    for idx, sample in enumerate(samples):
        cypher_query = cypher_queries[idx]
        # Execute query using Neo4j driver
        with neo4j_driver.session() as session:
            neo4j_result = session.run(cypher_query).data()
        # Use DSPy to match result (simulate with signature)
        response = executor(cypher=cypher_query)
        # Compare Neo4j result with DSPy result (for demonstration)
        match = neo4j_result == response.result
        results.append({
            "instance_id": sample['instance_id'],
            "cypher": cypher_query,
            "neo4j_result": neo4j_result,
            "dspy_result": response.result,
            "match": match
        })
    return results

# Example usage (requires a Neo4j driver instance):
# from neo4j import GraphDatabase
# neo4j_driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))
# results = execute_and_match(train_samples, cypher_queries_tamil, neo4j_driver)
# print(results[:5])